In [8]:
# Импортируем необходимые объекты
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import random
import json
import re
import pandas as pd
import sqlite3
from sqlite3 import Error
import os 
import datetime

In [9]:
# ------- Этот блок содержит глобальные переменные, необходимые для работы --------- #

#Далее создается элемент класса Firefox WebDriver для нашего url
#Единый указатель ресурса (англ. Uniform Resource Locator, URL 
# — единообразный локатор (определитель местонахождения) ресурса (в нашем случае сайт Картотеки арбитражных дел)
url = 'http://ras.arbitr.ru/'  
path_to_user_driver = 'C:/Users/besso/Desktop/VKR/chromedriver.exe'
path_to_anticaptcha_plugin = 'C:/Users/besso/Desktop/VKR/anticaptcha-plugin_v0.3410.zip' # Имеется в виду Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали
anti_captcha_key = '3d8da2d26ff3ba9552e5524629cface8'



In [10]:
#Параметры, которые должны быть забиты из приложения !

date_from = ''
date_to = ''
doc_text_input = 'казенное учреждение'
download_directory = 'C:\\Users\\besso\\Desktop\\VKR\\'

#собственно сюда нужно забить необходимые параметры

#C:\Users\besso\Desktop\VKR

In [11]:
# Метод отправки API запроса прямо в плагин
# Например для инициализации API ключа сервиса anti-captcha.com, необходимый для работы плагина
# Работает только на действующей HTML страничке,
# в нашем случае на https://antcpt.com/blank.html
# на страницах вроде about:blank запрос не пройдет
def acp_api_send_request(driver, message_type, data={}):
    message = {
        # всегда указывается именно этот получатель API сообщения
        'receiver': 'antiCaptchaPlugin',
        # тип запроса, например setOptions
        'type': message_type,
        # мерджим с дополнительными данными
        **data
    }
    # выполняем JS код на странице
    # а именно отправляем сообщение стандартным методом window.postMessage
    return driver.execute_script("""
    return window.postMessage({});
    """.format(json.dumps(message)))


In [12]:
def init_chrome_options(path_to_anticaptcha_plugin):
    # Инциируем объект опций для Хрома, чтобы иметь возможность подключить расширение
    options = webdriver.ChromeOptions()
    # Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали ранее
    options.add_extension(path_to_anticaptcha_plugin)
    prefs = {"plugins.always_open_pdf_externally": True, 
            'download.default_directory': download_directory} #Без дополнительных прожатий кнопки "скачать" при открытии pdf-скачиваем его
    options.add_experimental_option("prefs",prefs)
    return options

In [13]:
# Ф-ция для инцииализации веб-драйвера хрома опциями
def get_browser_with_options(path_to_user_driver):
    # Запускаем Браузер (Веб Драйвер Хрома) с указанием места скачивания самого файла драйвера
    browser = webdriver.Chrome(path_to_user_driver, options=init_chrome_options(path_to_anticaptcha_plugin) )
    return browser

In [14]:
# Ф-ция для получения стартовой страницы и обхода капчи на ней
def get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ):
    # Переходим на пустую страницу для выполнения API запроса к плагину
    browser = get_browser_with_options(path_to_user_driver)
    
    browser.get(url)
    
    #здесь делаем закрытие объявления (в случае, если оно появилось)
    #если объявление на сайте не появилось, то Selenium просто не найдет объект, отвечающий за него,
    #и при попытке закрыть объявление сгенерируется исключение
    #чтобы из-за этого исключения не падал весь дальнейший код, делаем обработку исключения с возможностью
    #выполнения остатка кода
    
    try:
        button_close = browser.find_element_by_class_name('b-promo_notification-popup-close')
        button_close.click()
    except NoSuchElementException:
        pass
    
    # Устанавливаем API ключ anti-captcha.com
    # замените YOUR-ANTI-CAPTCHA-API-KEY на Ваш шестнадцатиричный ключ, который можно взять тут:
    # https://anti-captcha.com/clients/settings/apisetup
    
    acp_api_send_request(
        browser,
        'setOptions',
        {'options': {'antiCaptchaApiKey': anti_captcha_key}}
    )

    # Три секунды паузы чтобы плагин проверил ключ на стороне anti-captcha.com
    time.sleep(10)
    
    return browser



In [15]:
# Необходимо добавить функцию, которая будет колбасить курсором рандомно во избежание бана

In [16]:
def human_text_enter( element, text_input ):
    time.sleep(random.uniform(2, 5)) 
    # sending_text = 'мошенничество'  text input 
    #document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]') 
    for letter in text_input: 
        time.sleep(random.uniform(0.05, 0.16)) 
        element.send_keys(letter)
    

In [17]:
# Ф-ция, которая забивает форму на сайте нужными нам параметрами (пока параметры не все, потом добавим, сейчас в тестовом режиме)
def get_page_by_input_attributes(browser= get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ), doc_text_in = '', date_from_in = '', date_to_in = '' ):
    
    document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]') #текст документа
   # document_text.send_keys(doc_text_in)
    
    human_text_enter(document_text, doc_text_in) # Эмулируем ввод текста документа человеком
    
    #dispute_type = browser.find_element_by_xpath('//input[@placeholder="вид спора"]') #вид спора
    #dispute_category = browser.find_element_by_xpath('//input[@placeholder="категория спора"]') #категория спора
    
    #case_participant = browser.find_element_by_xpath('//textarea[@placeholder="название, ИНН или ОГРН"]') #название, ИНН или ОГРН
    #court = browser.find_element_by_xpath('//input[@placeholder="название суда"]') #название суда
    #court.send_keys(court_in)
    #case_number = browser.find_element_by_xpath('//input[@placeholder="например, А50-5568/08"]') #номер дела
    
    date_from = browser.find_element_by_xpath('//label[@class="from"]//input[@placeholder="дд.мм.гггг"]') #нижняя граница даты
    human_text_enter(date_from, date_from_in) # Эмулируем ввод дат человеком
    
    date_to = browser.find_element_by_xpath('//label[@class="to"]//input[@placeholder="дд.мм.гггг"]') #верхняя граница даты
    human_text_enter(date_to, date_to_in)
    
    search_button = browser.find_element_by_xpath('//button[@alt="Найти"]') #кнопка "Найти"
    
    # Самая важная чаcть: ждем не более 120 секунд пока индикатор антикаптчи с классом antigate_solver
    # не получит класс solved, что означает что рекапча решена
    WebDriverWait(browser, 120)
    
    # Отправляем форму
    search_button.click()
    
    time.sleep(10)
    #ищем ссылку для перехода на страницу документа и переходим по ней
    return browser


In [18]:
initial_parsing_page = get_page_by_input_attributes(doc_text_in =  doc_text_input ) # пока это текущий браузер

In [19]:
def get_row_links_from_1_page(fist_page=initial_parsing_page):
    links = initial_parsing_page.find_elements_by_xpath("//a[@href]")
    all_links = list()
    for link in links:
        all_links.append(link.get_attribute("href"))
    return all_links
   

In [20]:
def get_pdf_links(all_links): 
    pdf_links = list()
    for link in all_links:
        if '.pdf' in link:
            pdf_links.append(link)
        else:
            continue
    return pdf_links


In [21]:
def count_found_documents(intial_page = initial_parsing_page):
    total_count = initial_parsing_page.find_elements_by_xpath('//*[@id="contentHeader"]/span[1]')
    split_text = total_count[0].text.split()
    return int(split_text[1])

In [22]:
def current_doc_info_text(intial_page, n  ):
    pattern_searh = '//*[@id="b-cases"]/li['
    pattern_searh = pattern_searh + str(n) + ']'
    try:
        current_page = initial_parsing_page.find_elements_by_xpath(pattern_searh) # будем итерироваться по индексу
        description = current_page[0].text
    except:
        NoSuchElementException
        pass
    return description

#можем выцепить порядковый номер дела, Номер дела, Инстанция, дата размещения документа , название документа

In [23]:
# Получаем данные для заполнения бд 
def get_doc_attributes(description ):
    ordinal_doc_number = int(description[0 : description.find('.') ]) # порядковый номер дела (нужен для показа загруженных материалов)
    start_pos_case_number = description.find('\n')
    end_pos_case_number = description.find('\n', start_pos_case_number + 1 ) # ищет второе вхождение этого символа в строке
    case_number = description[start_pos_case_number + 1 : end_pos_case_number ] # Номер дела
    try:
        date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[1] # дата заливки документа
    except:
        IndexError # индекс не входит в диапазон элементов.
        date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[0] # дата заливки документа
    
    court_name = description[end_pos_case_number + 1 : description.find(date_doc)] # Название суда, выносившего решение

    start_doc_pos = description.find(date_doc) + 10
    doc_name = description[ start_doc_pos : ]
    indexs = [i for i, symb in enumerate(doc_name) if symb=='\n']
    document_name = '' # название документа
    for i in range(len(doc_name)):
        if i not in indexs:
            document_name +=  doc_name[i] 
    
    return ordinal_doc_number, case_number, date_doc, court_name, document_name

    

In [24]:
# Скачиваются все файлы с текущей страницы браузера. Доработать так, чтобы итерировалось еще по другим страницам
def download_description(browser, pdf_links ):
    
    insert_values = { 'ordinal_number' : [], 
                      'case_number' : [],
                      'date_doc' : [], 
                      'court_name' : [],
                      'document_name' : [],
                      'pdf_link' : [],
                    }
    
    cnt_documents = count_found_documents() # Общее количество документов
    
    if cnt_documents > 40 * 25:
        cnt_documents = 40 * 25 # Сайт выдает только 40 страниц по 25 документов 
    
    for i in range(len(pdf_links)):
        
        description = current_doc_info_text(browser, i + 1 ) # получили описание нужного дока 
        
        ordinal_doc_number, case_number, date_doc, court_name, document_name = get_doc_attributes(description = description)
        
        
        # добавить в словарь key - value и расширять его по ключам 
        
        
        print("Идет загрузка документа {0} из {1}".format(ordinal_doc_number, cnt_documents))
        print(ordinal_doc_number)
        if pdf_links[i] not in insert_values['pdf_link']:
            insert_values['ordinal_number'].append(ordinal_doc_number)
            print(case_number)
            insert_values['case_number'].append(case_number)
            print(date_doc)
            insert_values['date_doc'].append(date_doc)
            print(court_name)
            insert_values['court_name'].append(court_name)
            print(document_name)
            insert_values['document_name'].append(document_name)
            print(pdf_links[i]) # просто линк текущего документа
            insert_values['pdf_link'].append(pdf_links[i])
        #time.sleep(random.uniform(2, 3))  - раскомментить
        else:
            continue
    return insert_values
       
    
    
   

In [65]:
def parse_all_documents(initial_parsing_page):
    cnt_documents = count_found_documents(initial_parsing_page) # Общее количество документов
    # Проход по всем страницам 
    flag = True
    i = 1 # для теста нужно заменить, чтобы было не с 1 страницы 
    x_path = '//*[@id="pages"]/li'
    current_page = initial_parsing_page
    links_need_to_be_downloaded = list()
    list_of_insert_values = list()
    while flag and i <= 12 : # убрать вторую часть условия для того, чтобы был парсинг ВСЕХ  страниц 
            print(i)
            if i < 10:
                try :
                    #print("page nom ", i)
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i + 2) + ']/a') # После парсинга текущей страницы переходим к следующей
                    #print(x_path + '[' + str(i + 1) + ']/a')
                   # print("ya tut 1 ")
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page = success")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links = success")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    list_of_insert_values.append(insert_values)
                    #download_docs(browser, pdf_links)
                    
                    next_next_page = next_page.click()
                    #print(x_path + '[' + str(i + 1) + ']/a')
                    i += 1 
                    time.sleep(random.uniform(5, 10))         
                except : 
                    NoSuchElementException
                    flag = False                    
                    
            elif i % 10 == 0:
                try:
                    if i == 10: 
                        j = 11
                    else :
                        j = 13 

                
                    #print(x_path + '[' + str(i + 1) + ']/a' + 'elif')
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(j) + ']/a') # После парсинга текущей страницы переходим к следующей
                
                    
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page- succ")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links - succ ")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    #print("download_description - succ ")
                    list_of_insert_values.append(insert_values) 
                    #print("append - succ ")
                    
                    #download_docs(browser, pdf_links)
                    
                    next_next_page = next_page.click()
                    #print(x_path + '[' + str(j) + ']/a')
                    i += 1 
                    time.sleep(random.uniform(5, 10))          
                    next_page = initial_parsing_page.find_element_by_xpath(x_path + '[' + str(j + 1 ) + ']/a') # После парсинга текущей страницы переходим к следующей
                    next_page.click()
                    time.sleep(random.uniform(5, 10))  

                except:
                    NoSuchElementException
                    flag = False
            else:
                try:
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i % 10  + 3)  + ']/a') # После парсинга текущей страницы переходим к следующей
                    #print("np=", next_page)
                    #print("ya tut ")
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page - else block  ")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links - else block  ")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    #print("download_description - else block  ")
                    
                    # добавить проверку на то, что если есть такая ссылка, то не добавлять ее в список!
                    
                    
                    list_of_insert_values.append(insert_values)
                    #print("append - else block  ")
                    #download_docs(browser, pdf_links)
                    
                    next_page.click()
                    #print("click - else block  ")
                    i += 1 
                    time.sleep(random.uniform(5, 10))        
                except : 
                    NoSuchElementException
                    flag = False
    # Если мы здесь - значит следующую страницу найти нельзя ( значит надо обработать текущую, т.е. последннюю )
    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
    links_need_to_be_downloaded += pdf_links       
    insert_values = download_description(initial_parsing_page, pdf_links)
    list_of_insert_values.append(insert_values)
    initial_parsing_page.quit() 
                    
                    
          
        
    return links_need_to_be_downloaded, list_of_insert_values
                
                
                

In [27]:
all_needed_links, list_of_insert_values = parse_all_documents(initial_parsing_page=initial_parsing_page)
# Можно подавить вывод информации, которая появляется ниже, просто закомментив все print -ы, но для отладки так нагляднее

1
Идет загрузка документа 1 из 1000
1
А14-18572/2019
24.12.2020
АС Воронежской области 
РЕШЕНИЕ СУДА ПЕРВОЙ ИНСТАНЦИИИСК УДОВЛЕТВОРИТЬ ЧАСТИЧНО
https://ras.arbitr.ru/Kad/PdfDocument/56716b9d-e758-4823-82ca-ae8f08b396aa/f9b0f78a-0bac-4e11-b166-9660e60737bc/%D0%9014-18572-2019__20201224.pdf
Идет загрузка документа 2 из 1000
2
А58-1519/2019
10.04.2020
АС Республики Саха 
О ВКЛЮЧЕНИИ ТРЕБОВАНИЙ В РЕЕСТР ТРЕБОВАНИЙ КРЕДИТОРОВ
https://ras.arbitr.ru/Kad/PdfDocument/cb4c6355-8a99-4e2c-8c13-e9ea885ce633/0eccb44f-9e09-413d-8600-0fbf640b1f50/%D0%9058-1519-2019__20200410.pdf
Идет загрузка документа 3 из 1000
3
А58-823/2020
10.04.2020
АС Республики Саха 
РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМАТРИВАЕМОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВАИЗМЕНИТЬ РЕШЕНИЕ АДМИНИСТРАТИВНОГО ОРГАНА
https://ras.arbitr.ru/Kad/PdfDocument/7bd89435-756f-4c0d-8def-5e5f08c908a8/74c4487c-9ded-4d7f-9672-4dd2b406ece5/%D0%9058-823-2020__20200410.pdf
Идет загрузка документа 4 из 1000
4
А50-2011/2020
10.04.2020
АС Пермског

2
Идет загрузка документа 26 из 1000
26
А75-19649/2019
10.04.2020
8 арбитражный апелляционный суд 
ОСТАВИТЬ БЕЗ ИЗМЕНЕНИЯ РЕШЕНИЕ, А АПЕЛЛЯЦИОННУЮ ЖАЛОБУ - БЕЗ УДОВЛЕТВОРЕНИЯ (П.1 СТ.269 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/3a39822e-6b96-4a9e-b2f3-836573dde380/a286bb45-13de-4561-b3a5-250b94fa5f1b/%D0%9075-19649-2019__20200410.pdf
Идет загрузка документа 27 из 1000
27
А54-923/2020
10.04.2020
АС Рязанской области 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИСК ПОЛНОСТЬЮ ИЛИ ЧАСТИЧНО
https://ras.arbitr.ru/Kad/PdfDocument/e3466a46-bdcf-46d2-b155-811e4ca7e72b/a302fccb-ef4e-4c3e-83f8-9ed35729fc6b/%D0%9054-923-2020__20200410.pdf
Идет загрузка документа 28 из 1000
28
А67-13271/2018
10.04.2020
Верховный Суд РФ 
ОБ ОТКАЗЕ В ПЕРЕДАЧЕ КАССАЦИОННОЙ ЖАЛОБЫ (ПРЕДСТАВЛЕНИЯ) ДЛЯ РАССМОТРЕНИЯ В СУДЕБНОМ ЗАСЕДАНИИ СУДЕБНОЙ КОЛЛЕГИИ ВС РФ
https://ras.arbitr.ru/Kad/PdfDocument/33dbf44c-9e29-4af9-9a5c-8bdde971fb50/fca9da2f-ad2e-4dce-8347-32d23bd3ebcb/%D0%9067-13271-2018__20200410.pdf
Идет загрузка документа 29 из 1000


Идет загрузка документа 54 из 1000
54
А49-356/2020
10.04.2020
АС Пензенской области 
МОТИВИРОВАННОЕ РЕШЕНИЕ ПО ДЕЛУ, РАССМОТРЕННОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВА
https://ras.arbitr.ru/Kad/PdfDocument/37734d65-db75-4f97-800a-e87ab892b76b/70781756-0105-4ac9-ae3c-edd046816b85/%D0%9049-356-2020__20200410.pdf
Идет загрузка документа 55 из 1000
55
А40-128164/2019
10.04.2020
АС города Москвы 
[ПОДПИСАНО]РЕШЕНИЕ СУДА ПЕРВОЙ ИНСТАНЦИИИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
https://ras.arbitr.ru/Kad/PdfDocument/1b18c6c0-cf10-4210-9dcb-308c6af8046e/e7794bbe-85d9-4544-a691-240f9a04cc60/%D0%9040-128164-2019__20200410.pdf
Идет загрузка документа 56 из 1000
56
А50-7393/2020
10.04.2020
АС Пермского края 
О ВОЗВРАЩЕНИИ ЗАЯВЛЕНИЯ О ВЫДАЧЕ СУДЕБНОГО ПРИКАЗА
https://ras.arbitr.ru/Kad/PdfDocument/bd06dfea-e4f1-4b07-8b1f-36448ec8ad34/8689fc39-3744-4d7b-bcfa-8a902982c3cd/%D0%9050-7393-2020__20200410.pdf
Идет загрузка документа 57 из 1000
57
А34-1297/2020
10.04.2020
АС Курганской области 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИС

Идет загрузка документа 81 из 1000
81
А82-5824/2020
10.04.2020
АС Ярославской области 
[ПОДПИСАНО]ВЫДАТЬ СУДЕБНЫЙ ПРИКАЗ (СТ.229.6 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/04eb46a1-bf86-4530-9e0a-fb3221407b9c/e38d2d73-1d98-4e99-bfdf-82988cd49590/%D0%9082-5824-2020__20200410.pdf
Идет загрузка документа 82 из 1000
82
А82-5819/2020
10.04.2020
АС Ярославской области 
[ПОДПИСАНО]ВЫДАТЬ СУДЕБНЫЙ ПРИКАЗ (СТ.229.6 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/f8e9db29-7059-4164-87bc-287f1b2e010c/5d157e1b-7225-4adb-9129-cf839d1459a0/%D0%9082-5819-2020__20200410.pdf
Идет загрузка документа 83 из 1000
83
А04-2659/2020
10.04.2020
АС Амурской области 
[ПОДПИСАНО]ВОЗВРАТИТЬ ЗАЯВЛЕНИЕ О ВЫДАЧЕ СУДЕБНОГО ПРИКАЗА (Ч.2 СТ.229.4 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/0d08c3b5-f894-43d3-8cc4-2f625855d408/14c7d231-df81-48ac-a957-ec5484c5e557/%D0%9004-2659-2020__20200410.pdf
Идет загрузка документа 84 из 1000
84
А66-4318/2020
10.04.2020
АС Тверской области 
ВЫДАТЬ СУДЕБНЫЙ ПРИКАЗ (СТ.229.6 АПК)
https://r

Идет загрузка документа 110 из 1000
110
А83-4559/2020
09.04.2020
АС Республики Крым 
О ПРЕКРАЩЕНИИ ПРОИЗВОДСТВА ПО ДЕЛУ
https://ras.arbitr.ru/Kad/PdfDocument/4f3ab787-e61a-426a-8406-b5dcd16b4576/99b7dd3a-6a14-49de-bc1a-c11b521bf367/%D0%9083-4559-2020__20200409.pdf
Идет загрузка документа 111 из 1000
111
А82-19671/2019
09.04.2020
АС Ярославской области 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИСК ПОЛНОСТЬЮ ИЛИ ЧАСТИЧНОВОЗМЕСТИТЬ (РАСПРЕДЕЛИТЬ) СУДЕБНЫЕ РАСХОДЫ (СТ.101, 106, 112 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/91db16ee-0206-4695-8a9f-8cddf29b547b/c2a2bfc0-22a0-443b-9f27-3f158affd4a2/%D0%9082-19671-2019__20200409.pdf
Идет загрузка документа 112 из 1000
112
А65-1941/2020
09.04.2020
АС Республики Татарстан 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИСК ПОЛНОСТЬЮ ИЛИ ЧАСТИЧНО
https://ras.arbitr.ru/Kad/PdfDocument/0e4510a3-1644-4b9b-b1b3-a58ab368817f/49396206-3bd8-4895-a5c1-4d4c48ab16c4/%D0%9065-1941-2020__20200409.pdf
Идет загрузка документа 113 из 1000
113
А55-3159/2020
09.04.2020
АС Самарской области 
[ПОДПИСА

...

Идет загрузка документа 216 из 1000
216
А31-2361/2020
08.04.2020
АС Костромской области 
[ПОДПИСАНО]РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМАТРИВАЕМОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВАИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
https://ras.arbitr.ru/Kad/PdfDocument/2fffb2ca-747b-4553-98cf-0943a4e17337/1ba7ee80-b747-41be-b3e9-7ed14921f0c9/%D0%9031-2361-2020__20200408.pdf
Идет загрузка документа 217 из 1000
217
А75-2187/2020
08.04.2020
АС Ханты-Мансийского АО 
[ПОДПИСАНО]РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМАТРИВАЕМОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВАИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
https://ras.arbitr.ru/Kad/PdfDocument/4c7bb62f-9cdc-4947-8d89-778e835f1831/d19c2254-aa8e-4441-a122-64e54f45f0ae/%D0%9075-2187-2020__20200408.pdf
Идет загрузка документа 218 из 1000
218
А58-875/2020
08.04.2020
АС Республики Саха 
РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМАТРИВАЕМОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВАИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
https://ras.arbitr.ru/Kad/PdfDocument/26e86425-65db-4225-ba9e-a92

Идет загрузка документа 241 из 1000
241
А56-26188/2019
08.04.2020
Верховный Суд РФ 
ОБ ОТКАЗЕ В ПЕРЕДАЧЕ КАССАЦИОННОЙ ЖАЛОБЫ (ПРЕДСТАВЛЕНИЯ) ДЛЯ РАССМОТРЕНИЯ В СУДЕБНОМ ЗАСЕДАНИИ СУДЕБНОЙ КОЛЛЕГИИ ВС РФ
https://ras.arbitr.ru/Kad/PdfDocument/65ea0938-e172-408e-b2bf-a4b6f77b299e/4c0cc132-d7d6-420f-ab1a-70bad8cd218e/%D0%9056-26188-2019__20200408.pdf
Идет загрузка документа 242 из 1000
242
А12-7226/2020
08.04.2020
АС Волгоградской области 
О НЕВОЗМОЖНОСТИ ИСПОЛНЕНИЯ СУДЕБНОГО ПОРУЧЕНИЯ
https://ras.arbitr.ru/Kad/PdfDocument/1fbfd3d6-f5ae-41ee-a427-86258b51bf5c/ef9e2655-410d-41b3-9f03-c1b518ca554d/%D0%9012-7226-2020__20200408.pdf
Идет загрузка документа 243 из 1000
243
А55-15735/2017
08.04.2020
АС Самарской области 
[ПОДПИСАНО]ИЗМЕНИТЬ ДАТУ СУДЕБНОГО ЗАСЕДАНИЯ ПО РАССМОТРЕНИЮ ХОДАТАЙСТВА (ЗАЯВЛЕНИЯ, ЗАПРОСА) (ПО АНАЛОГИИ СО СТ.158 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/9626e297-05e3-4a3f-bd01-57338d3aacd1/e7fe0981-0918-487f-8908-995bd5ef5cc4/%D0%9055-15735-2017__20200408.pdf
Идет загрузк

Идет загрузка документа 270 из 1000
270
А03-4017/2020
07.04.2020
АС Алтайского края 
СУДЕБНЫЙ ПРИКАЗИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
https://ras.arbitr.ru/Kad/PdfDocument/1c84bc19-60ca-455b-8e9e-f8e2476bc04b/d095b8f1-8c67-472c-8eb5-e432db19d030/%D0%9003-4017-2020__20200407.pdf
Идет загрузка документа 271 из 1000
271
А76-53781/2019
07.04.2020
АС Челябинской области 
МОТИВИРОВАННОЕ РЕШЕНИЕ ПО ДЕЛУ, РАССМОТРЕННОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВА
https://ras.arbitr.ru/Kad/PdfDocument/53a640db-f545-468f-8091-64fd146631d5/ebf8645e-3f3d-4015-b6cd-a84b6afc92e4/%D0%9076-53781-2019__20200407.pdf
Идет загрузка документа 272 из 1000
272
А56-89599/2019
07.04.2020
АС Северо-Западного округа 
ОСТАВИТЬ БЕЗ ИЗМЕНЕНИЯ РЕШЕНИЕ И (ИЛИ) ПОСТАНОВЛЕНИЕ АПЕЛЛЯЦИОННОЙ ИНСТАНЦИИ, А КАССАЦИОННУЮ ЖАЛОБУ - БЕЗ УДОВЛЕТВОРЕНИЯ (П.1 Ч.1 СТ.287 АПК)
https://ras.arbitr.ru/Kad/PdfDocument/1952424d-729a-4251-b054-da4bca41d1d4/59cce37e-a3c2-402c-b813-8e8974d94b9a/%D0%9056-89599-2019__20200407.pdf
Идет загрузка документа 273

Идет загрузка документа 273 из 1000
273
А05-11079/2019
07.04.2020
Верховный Суд РФ 
ОБ ОТКАЗЕ В ПЕРЕДАЧЕ КАССАЦИОННОЙ ЖАЛОБЫ (ПРЕДСТАВЛЕНИЯ) ДЛЯ РАССМОТРЕНИЯ В СУДЕБНОМ ЗАСЕДАНИИ СУДЕБНОЙ КОЛЛЕГИИ ВС РФ
https://ras.arbitr.ru/Kad/PdfDocument/bdeb472a-6475-432a-8f0a-2b2a1d9efbd8/5cd5594c-efdc-428d-b95b-d67c822e5c8e/%D0%9005-11079-2019__20200407.pdf
Идет загрузка документа 274 из 1000
274
А56-28209/2018
07.04.2020
Верховный Суд РФ 
ОБ ОТКАЗЕ В ПЕРЕДАЧЕ КАССАЦИОННОЙ ЖАЛОБЫ (ПРЕДСТАВЛЕНИЯ) ДЛЯ РАССМОТРЕНИЯ В СУДЕБНОМ ЗАСЕДАНИИ СУДЕБНОЙ КОЛЛЕГИИ ВС РФ
https://ras.arbitr.ru/Kad/PdfDocument/d66b0e58-47fe-48a1-ab52-7a1c1dbf7848/b91ac256-de01-4c34-8cd6-8180b69d9724/%D0%9056-28209-2018__20200407.pdf
Идет загрузка документа 275 из 1000
275
А56-13531/2020
07.04.2020
АС города Санкт-Петербурга и Ленинградской области 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИСК ПОЛНОСТЬЮ ИЛИ ЧАСТИЧНО
https://ras.arbitr.ru/Kad/PdfDocument/42d4e58d-5f20-4008-b08f-82333b249ff6/91176f40-5b80-4c1c-9b33-9a0063acea8f/%D0%9056-13531-2020_

Идет загрузка документа 299 из 1000
299
А24-7511/2019
07.04.2020
АС Камчатского края 
[ПОДПИСАНО]ИЗМЕНИТЬ ДАТУ СУДЕБНОГО ЗАСЕДАНИЯ ПО РАССМОТРЕНИЮ ТРЕБОВАНИЯ КРЕДИТОРА (ПО АНАЛОГИИ СО СТ.158 АПК, СТ.71, СТ.100 ФЗ О НЕСОСТОЯТЕЛЬНОСТИ)
https://ras.arbitr.ru/Kad/PdfDocument/e09eece8-f7f0-4882-84fa-646a0518501e/f561fc80-c6af-4e75-bddf-960c9d3bf0e2/%D0%9024-7511-2019__20200407.pdf
Идет загрузка документа 300 из 1000
300
А65-37678/2019
07.04.2020
АС Республики Татарстан 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИСК ПОЛНОСТЬЮ ИЛИ ЧАСТИЧНО
https://ras.arbitr.ru/Kad/PdfDocument/2f5da2ef-b685-4f87-a9c4-9b0210284541/6d21d0d4-eecb-4487-9f24-fd290acb450f/%D0%9065-37678-2019__20200407.pdf


In [28]:
res_insert_values = pd.concat([pd.DataFrame(d) for d in list_of_insert_values]).to_dict('list')


In [30]:
def data_frame(res_insert_values):
    return pd.DataFrame.from_dict(res_insert_values)

In [31]:
df = data_frame(res_insert_values)


In [32]:
df

,ordinal_number,case_number,date_doc,court_name,document_name,pdf_link
0,1,А14-18572/2019,24.12.2020,АС Воронежской области,РЕШЕНИЕ СУДА ПЕРВОЙ ИНСТАНЦИИИСК УДОВЛЕТВОРИТЬ...,https://ras.arbitr.ru/Kad/PdfDocument/56716b9d...
1,2,А58-1519/2019,10.04.2020,АС Республики Саха,О ВКЛЮЧЕНИИ ТРЕБОВАНИЙ В РЕЕСТР ТРЕБОВАНИЙ КРЕ...,https://ras.arbitr.ru/Kad/PdfDocument/cb4c6355...
2,3,А58-823/2020,10.04.2020,АС Республики Саха,"РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМ...",https://ras.arbitr.ru/Kad/PdfDocument/7bd89435...
3,4,А50-2011/2020,10.04.2020,АС Пермского края,"РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМ...",https://ras.arbitr.ru/Kad/PdfDocument/fc3efdb0...
4,5,А53-3608/2020,10.04.2020,АС Ростовской области,"РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМ...",https://ras.arbitr.ru/Kad/PdfDocument/385b6295...
...,...,...,...,...,...,...
320,296,А70-22702/2019,07.04.2020,АС Тюменской области,[ПОДПИСАНО]ПРИНЯТЬ К РАССМОТРЕНИЮ ХОДАТАЙСТВО ...,https://ras.arbitr.ru/Kad/PdfDocument/30612ba1...
321,297,А56-24429/2020,07.04.2020,АС города Санкт-Петербурга и Ленинградской обл...,[ПОДПИСАНО]ВОЗВРАТИТЬ ЗАЯВЛЕНИЕ (ИСКОВОЕ ЗАЯВЛ...,https://ras.arbitr.ru/Kad/PdfDocument/b3631b04...
322,298,А76-28679/2018,07.04.2020,Верховный Суд РФ,ОБ ОТКАЗЕ В ПЕРЕДАЧЕ КАССАЦИОННОЙ ЖАЛОБЫ (ПРЕД...,https://ras.arbitr.ru/Kad/PdfDocument/9c0fbde3...
323,299,А24-7511/2019,07.04.2020,АС Камчатского края,[ПОДПИСАНО]ИЗМЕНИТЬ ДАТУ СУДЕБНОГО ЗАСЕДАНИЯ П...,https://ras.arbitr.ru/Kad/PdfDocument/e09eece8...


In [ ]:
# Создание схемы бд  

In [56]:
name_db = 'my_vkr.db'

In [57]:
def sql_connection(db_name = name_db):
    try:
        con = sqlite3.connect(name_db) # устанавливаем соединение с БД
        return con
    except Error:
        print(Error)
        

In [58]:
con = sql_connection()

In [59]:
def create_tables(connection = con):
    cursor_obj = con.cursor()
    cursor_obj.execute(""" CREATE TABLE SEARCH_PARAMS(
                           ID_DOC INTEGER,  
                           SEARCH_DIRECTION TEXT,
                           DATE_FROM TEXT, 
                           DATE_TO TEXT,
                           DOWNLOAD_DATE TEXT,
                           LOCATED_IN_DIRECTORY TEXT ) """)
    con.commit()
    
    cursor_obj = con.cursor()
    cursor_obj.execute(""" CREATE TABLE DOC_INFO(
                           ID_DOC INTEGER, 
                           CASE_NUMBER TEXT, 
                           DATE_ON_SITE TEXT, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                           COURT_NAME TEXT, 
                           DOC_TITLE TEXT, 
                           PDF_LINK TEXT) """)
    con.commit()
    
    
    # ЗДЕСЬ ЕЩЕ ДОЛЖНЫ ПОЯВИТЬСЯ 2 СУЩНОСТИ - ТАБЛИЦА С РЕЙТИНГАМИ ДОКОВ ( ПОСЛЕ ТОГО, КАК ОПРЕДЕЛИМСЯ, КАК ОЦЕНИВАЕМ)
    # И ТАБЛИЦА С ЮЗЕРАМИ, КОТОРЫЕ БУДУТ ИМЕТЬ ДОСТУП К БД ИЛИ ТИП ТОГО
    # надо обсудить этот момент
    
    con.close()
    

In [62]:
try :
    create_tables(connection = con) # создание табличек выдает ошибку если таблички уже существуют 
except:
    print("Table already exists")

Table already exists


In [ ]:
def add_constraints_on_tables(connection  = con):  # пока прототип будущей функции. Нужно согласовать, что будет PK 
    cursor_obj = con.cursor()
    cursor_obj.execute(""" ALTER TABLE SEARCH_PARAMS ADD CONSTRAINT ..... """) 
    
    con.commit()
    
    cursor_obj.execute(""" ALTER TABLE DOC_INFO ADD CONSTRAINT .... """)
    
    con.commit()
    
    con.close()
    
    # первичные ключи определить и еще мб какую-н сущность

In [ ]:
# нужен автоинкремент, чтобы при добавлении в таблицу взять след значение как первичный ключ

In [71]:
def download_docs( browser, ins_vals,  connection = con):
    cursor_obj = con.cursor()
    #browser = get_initial_page()
    n = len(ins_vals['ordinal_number'])
    for i in range(n):
        print("Загрузка документа {0} из {1}".format(i + 1, n ))
        browser.get(ins_vals['pdf_link'][i]) # скачиваем документ по ссылке 
        
        cursor_obj.execute("""
        INSERT INTO DOC_INFO(
                    ID_DOC, 
                    CASE_NUMBER, 
                    DATE_ON_SITE, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                    COURT_NAME, 
                    DOC_TITLE, 
                    PDF_LINK)  
        VALUES (?, ?, ?, ?, ?, ?)""", 
                ( ins_vals['ordinal_number'][i],
                  ins_vals['case_number'][i],
                  ins_vals['date_doc'][i],
                  ins_vals['court_name'][i],
                  ins_vals['document_name'][i],
                  ins_vals['pdf_link'][i]) )
        
        time.sleep(1)
        
        now = datetime.datetime.now()
        
        cursor_obj.execute("""
        INSERT INTO SEARCH_PARAMS(
                    ID_DOC,
                    SEARCH_DIRECTION, 
                    DATE_FROM, 
                    DATE_TO,  
                    DOWNLOAD_DATE, 
                    LOCATED_IN_DIRECTORY)
                    
        VALUES (?, ?, ?, ?, ?, ?)""", 
                ( ins_vals['ordinal_number'][i],
                  doc_text_input,
                  date_from,
                  date_to,
                  now.strftime("%d-%m-%Y %H:%M"),
                  download_directory) )
        
        
        time.sleep(random.uniform(5, 10))
        
        con.commit()
        
    browser.quit()
        
        # Заполнять еще таблицы, о которых упомяналось выше ( с рейтингами доков и юзерами ) 
    

In [72]:

browser = webdriver.Chrome(path_to_user_driver, options=init_chrome_options(path_to_anticaptcha_plugin) )

download_docs( browser =  browser , ins_vals = res_insert_values ,  connection = con  ) # загрузка доков в указанную папку


Загрузка документа 1 из 325
Загрузка документа 2 из 325


KeyboardInterrupt: 